## Demonstration for the `u_mass` topic coherence using topic coherence pipeline

In [1]:
import numpy as np
import logging
import pyLDAvis.gensim
import json
import warnings
warnings.filterwarnings('ignore')  # To ignore all warnings that arise here to enhance clarity

from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
from numpy import array

### Set up logging

In [2]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

### Set up corpus

As stated in table 2 from [this](http://www.cs.bham.ac.uk/~pxt/IDA/lsa_ind.pdf) paper, this corpus essentially has two classes of documents. First five are about human-computer interaction and the other four are about graphs. Let's see how our LDA models interpret them.

In [3]:
texts = [['human', 'interface', 'computer'],
         ['survey', 'user', 'computer', 'system', 'response', 'time'],
         ['eps', 'user', 'interface', 'system'],
         ['system', 'human', 'system', 'eps'],
         ['user', 'response', 'time'],
         ['trees'],
         ['graph', 'trees'],
         ['graph', 'minors', 'trees'],
         ['graph', 'minors', 'survey']]

In [4]:
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

### Set up two topic models

We'll be setting up two different LDA Topic models. A good one and bad one. To build a "good" topic model, we'll simply train it using more iterations than the bad one. Therefore the `u_mass` coherence should in theory be better for the good model than the bad one since it would be producing more "human-interpretable" topics.

In [5]:
goodLdaModel = LdaModel(corpus=corpus, id2word=dictionary, iterations=50, num_topics=2)
badLdaModel = LdaModel(corpus=corpus, id2word=dictionary, iterations=1, num_topics=2)

In [6]:
goodcm = CoherenceModel(model=goodLdaModel, corpus=corpus, dictionary=dictionary, coherence='u_mass')

In [7]:
badcm = CoherenceModel(model=badLdaModel, corpus=corpus, dictionary=dictionary, coherence='u_mass')

### View the pipeline parameters for one coherence model

Following are the pipeline parameters for `u_mass` coherence. By pipeline parameters, we mean the functions being used to calculate segmentation, probability estimation, confirmation measure and aggregation as shown in figure 1 in [this](http://svn.aksw.org/papers/2015/WSDM_Topic_Evaluation/public.pdf) paper.

In [8]:
print goodcm

CoherenceModel(segmentation=<function s_one_pre at 0x7fcfdbafe050>, probability estimation=<function p_boolean_document at 0x7fcfdbafe320>, confirmation measure=<function log_conditional_probability at 0x7fcfdbafe488>, aggregation=<function arithmetic_mean at 0x7fcfdbafe410>)


### Visualize topic models

In [9]:
pyLDAvis.enable_notebook()

In [10]:
pyLDAvis.gensim.prepare(goodLdaModel, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x    y
topic                                           
0      53.784345        1       1 -0.042457 -0.0
1      46.215655        1       2  0.042457 -0.0, topic_info=     Category      Freq       Term     Total  loglift  logprob
term                                                          
2     Default  3.000000     system  3.000000  12.0000  12.0000
4     Default  2.000000        eps  2.000000  11.0000  11.0000
10    Default  2.000000  interface  2.000000  10.0000  10.0000
8     Default  2.000000      human  2.000000   9.0000   9.0000
1     Default  2.000000      graph  2.000000   8.0000   8.0000
3     Default  2.000000      trees  2.000000   7.0000   7.0000
0     Default  2.000000     minors  2.000000   6.0000   6.0000
11    Default  2.000000   response  2.000000   5.0000   5.0000
9     Default  2.000000       time  2.000000   4.0000   4.0000
6     Default  2.000000     survey  2.000000   3.0000   3.0000
5     Default  2.000000   computer  2.000000   2.0000   2.0000
7     Default  2.000000       user  2.000000   1.0000   1.0000
1      Topic1  2.160895      graph  2.887064   0.3305  -1.9766
3      Topic1  2.155307      trees  2.886559   0.3281  -1.9792
0      Topic1  1.602852     minors  2.163687   0.3202  -2.2753
11     Topic1  1.494527   response  2.153892   0.2547  -2.3453
9      Topic1  1.477177       time  2.152323   0.2438  -2.3570
6      Topic1  1.394609     survey  2.144858   0.1897  -2.4145
7      Topic1  1.644094       user  2.840335   0.0735  -2.2499
5      Topic1  1.058064   computer  2.114427  -0.0722  -2.6907
8      Topic1  0.665682      human  2.078948  -0.5186  -3.1541
10     Topic1  0.617191  interface  2.074563  -0.5921  -3.2297
4      Topic1  0.500226        eps  2.063987  -0.7971  -3.4398
2      Topic1  0.826837     system  3.439357  -0.8052  -2.9373
2      Topic2  2.612520     system  3.439357   0.4969  -1.6351
4      Topic2  1.563761        eps  2.063987   0.4943  -2.1484
10     Topic2  1.457372  interface  2.074563   0.4187  -2.2188
8      Topic2  1.413266      human  2.078948   0.3859  -2.2495
5      Topic2  1.056363   computer  2.114427   0.0779  -2.5406
7      Topic2  1.196241       user  2.840335  -0.0929  -2.4163
6      Topic2  0.750248     survey  2.144858  -0.2786  -2.8828
9      Topic2  0.675147       time  2.152323  -0.3875  -2.9883
11     Topic2  0.659366   response  2.153892  -0.4119  -3.0119
0      Topic2  0.560835     minors  2.163687  -0.5783  -3.1738
3      Topic2  0.731252      trees  2.886559  -0.6012  -2.9084
1      Topic2  0.726170      graph  2.887064  -0.6084  -2.9154, token_table=      Topic      Freq       Term
term                            
5         1  0.472941   computer
5         2  0.472941   computer
4         1  0.484499        eps
4         2  0.968998        eps
1         1  0.692745      graph
1         2  0.346373      graph
8         1  0.481013      human
8         2  0.481013      human
10        1  0.482029  interface
10        2  0.482029  interface
0         1  0.924348     minors
0         2  0.462174     minors
11        1  0.464276   response
11        2  0.464276   response
6         1  0.466231     survey
6         2  0.466231     survey
2         1  0.290752     system
2         2  0.872256     system
9         1  0.464614       time
9         2  0.464614       time
3         1  0.692866      trees
3         2  0.346433      trees
7         1  0.704142       user
7         2  0.352071       user, R=12, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2])

In [11]:
pyLDAvis.gensim.prepare(badLdaModel, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x    y
topic                                           
0      54.719704        1       1 -0.003725 -0.0
1      45.280296        1       2  0.003725 -0.0, topic_info=     Category      Freq       Term     Total  loglift  logprob
term                                                          
3     Default  2.000000      trees  2.000000  12.0000  12.0000
7     Default  2.000000       user  2.000000  11.0000  11.0000
9     Default  2.000000       time  2.000000  10.0000  10.0000
1     Default  2.000000      graph  2.000000   9.0000   9.0000
11    Default  2.000000   response  2.000000   8.0000   8.0000
6     Default  2.000000     survey  2.000000   7.0000   7.0000
10    Default  2.000000  interface  2.000000   6.0000   6.0000
2     Default  3.000000     system  3.000000   5.0000   5.0000
0     Default  2.000000     minors  2.000000   4.0000   4.0000
5     Default  2.000000   computer  2.000000   3.0000   3.0000
4     Default  2.000000        eps  2.000000   2.0000   2.0000
8     Default  2.000000      human  2.000000   1.0000   1.0000
6      Topic1  1.403463     survey  2.166565   0.1687  -2.4254
5      Topic1  1.323382   computer  2.151822   0.1168  -2.4842
4      Topic1  1.283536        eps  2.144487   0.0897  -2.5147
8      Topic1  1.260209      human  2.140192   0.0733  -2.5331
0      Topic1  1.208988     minors  2.130763   0.0362  -2.5746
2      Topic1  2.003470     system  3.549152   0.0311  -2.0695
10     Topic1  1.169065  interface  2.123413   0.0061  -2.6081
11     Topic1  1.124464   response  2.115202  -0.0289  -2.6470
1      Topic1  1.497503      graph  2.819941  -0.0300  -2.3606
7      Topic1  1.420522       user  2.805769  -0.0777  -2.4133
9      Topic1  1.045865       time  2.100732  -0.0945  -2.7195
3      Topic1  1.128246      trees  2.751962  -0.2887  -2.6437
3      Topic2  1.623715      trees  2.751962   0.2647  -2.0903
9      Topic2  1.054867       time  2.100732   0.1034  -2.5216
7      Topic2  1.385247       user  2.805769   0.0865  -2.2491
1      Topic2  1.322438      graph  2.819941   0.0351  -2.2955
11     Topic2  0.990738   response  2.115202   0.0338  -2.5843
10     Topic2  0.954348  interface  2.123413  -0.0075  -2.6217
2      Topic2  1.545682     system  3.549152  -0.0389  -2.1395
0      Topic2  0.921775     minors  2.130763  -0.0456  -2.6565
8      Topic2  0.879983      human  2.140192  -0.0965  -2.7029
4      Topic2  0.860950        eps  2.144487  -0.1203  -2.7247
5      Topic2  0.828441   computer  2.151822  -0.1622  -2.7632
6      Topic2  0.763102     survey  2.166565  -0.2512  -2.8454, token_table=      Topic      Freq       Term
term                            
5         1  0.464722   computer
5         2  0.464722   computer
4         1  0.466312        eps
4         2  0.466312        eps
1         1  0.354617      graph
1         2  0.354617      graph
8         1  0.467248      human
8         2  0.467248      human
10        1  0.470940  interface
10        2  0.470940  interface
0         1  0.469316     minors
0         2  0.469316     minors
11        1  0.472768   response
11        2  0.472768   response
6         1  0.461560     survey
6         2  0.461560     survey
2         1  0.563515     system
2         2  0.563515     system
9         1  0.476025       time
9         2  0.476025       time
3         1  0.363377      trees
3         2  0.726754      trees
7         1  0.356409       user
7         2  0.356409       user, R=12, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2])

In [12]:
print goodcm.get_coherence()

-13.8048438862


In [13]:
print badcm.get_coherence()

-15.5467907012


### Conclusion

Hence as we can see, the `u_mass` coherence for the good LDA model is much more (better) than that for the bad LDA model. This is because, simply, the good LDA model usually comes up with better topics that are more human interpretable.
For the first topic, the goodLdaModel rightly puts emphasis on "graph", "trees" and "user" with reference to the second class of documents.
For the second topic, it puts emphasis on words such as "system", "eps", "interface" and "human" which signify human-computer interaction.
The badLdaModel however fails to decipher between these two topics and comes up with topics which are mostly both graph based but are not clear to a human. The `u_mass` topic coherence captures this wonderfully by giving the interpretability of these topics and number as we can see above. Hence this coherence measure can be used to compare difference topic models based on their human-interpretability.